ELEC6910X Advanced Topics in AI and Healthcare

# Assignment 1

***Holy Lovenia - 20814158***

------

In [1]:
from torch.utils.data import Dataset
from torchvision import datasets

import h5py
import os
import pandas as pd
import torch

## Problem 1

In this problem, you are required to implement a 3D segmentation network (e.g., 3d unet), including
the model, dataloader, training/testing.

In [5]:
class Problem1Dataset(Dataset):
    def __init__(self, dir_path, column_names=["image", "label"], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dir_path = dir_path
        self.column_names = column_names
        self._load_all_data_from_dir(dir_path)
    
    def _load_h5_data(self, file_path):
        hf = h5py.File(file_path, "r")
        return hf

    def _load_all_data_from_dir(self, dir_path):
        df_dict = {}
        for col in self.column_names:
            df_dict[col] = []

        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if os.path.isfile(file_path) and file_path.endswith(".h5"):
                data = self._load_h5_data(file_path)
                n_data = len(data[self.column_names[0]])
                for col in self.column_names:
                    df_dict[col] += [data[col][i] for i in range(n_data)]

        self.dataset = pd.DataFrame.from_dict(df_dict)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return self.dataset.iloc[index]